### Status: think the xyz coords are not consistent across monkeys
### Look at where the units are on a glass brain
- Load all the units for both monkeys
- figure out naming, color
- plot on glass brain
- (hope xyz coords are consistent across monkeys)


In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.behavioral_utils as behavioral_utils
import utils.information_utils as information_utils
import utils.visualization_utils as visualization_utils
import utils.pseudo_classifier_utils as pseudo_classifier_utils
import utils.classifier_utils as classifier_utils

import utils.io_utils as io_utils

import utils.glm_utils as glm_utils
from matplotlib import pyplot as plt
import matplotlib
import utils.spike_utils as spike_utils
import utils.subspace_utils as subspace_utils
from trial_splitters.condition_trial_splitter import ConditionTrialSplitter 
from utils.session_data import SessionData
from constants.behavioral_constants import *
from constants.decoding_constants import *
import seaborn as sns
from scripts.pseudo_decoding.belief_partitions.belief_partition_configs import *
import scripts.pseudo_decoding.belief_partitions.belief_partitions_io as belief_partitions_io

import scipy
import argparse
import copy
import plotly.express as px
from scripts.anova_analysis.anova_configs import *



In [2]:
def load_units_for_sub(subject):
    units = pd.read_pickle(UNITS_PATH.format(sub=subject))
    drift_units = pd.read_pickle(DRIFT_PATH.format(sub=subject))
    units = units[~units.PseudoUnitID.isin(drift_units.PseudoUnitID)]
    feats = pd.read_pickle(FEATS_PATH.format(sub=subject))
    sessions = feats.sessions.explode().unique()
    units = units[units.session.isin(sessions)]
    units = units.rename(columns={'structure_level2': 'Region'})
    units["Region"] = units["Region"].replace({": eexxttrraassttrriiaattee__vviissuuaall__aarreeaass__22--44 ((VV22--VV44))": "extrastriate_visual_areas_2-4 (V2-V4)"})
    units["Subject"] = subject
    return units

In [6]:
sa_units = load_units_for_sub("SA")
bl_units = load_units_for_sub("BL")
# all_units = pd.concat((sa_units, bl_units))
all_units = sa_units
all_units["region_formatted"] = all_units.structure_level2_cleaned.apply(lambda x: visualization_utils.REGION_TO_DISPLAY_NAMES[x] if x in visualization_utils.REGION_TO_DISPLAY_NAMES else "other")


In [8]:
name_to_color = copy.deepcopy(visualization_utils.REGION_TO_COLOR)
name_to_color["other"] = "grey"

fig = visualization_utils.generate_glass_brain(all_units, "region_formatted", name_to_color=name_to_color, unknown_color="grey")
fig.write_html("/data/patrick_res/figures/wcst_paper/units/glass_brain.html")
# fig.show()

In [9]:
sa_units.session.nunique()

44

In [10]:
bl_units.session.nunique()

30